In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

# Load tokenizer and add special token if missing
tokenizer = AutoTokenizer.from_pretrained("sarvamai/sarvam-1", trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Load the base model
base_model = AutoModelForCausalLM.from_pretrained("sarvamai/sarvam-1", trust_remote_code=True)

# Resize the model embeddings to match the tokenizer’s vocabulary size
base_model.resize_token_embeddings(len(tokenizer))

# Load the fine-tuned adapter
model = PeftModel.from_pretrained(base_model, "./final_model")

# Set device (GPU if available, else CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define your prompt
prompt = "Write a short Indian Folklore with moral"

# Tokenize the prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

# Generate text with specified parameters
generated_ids = model.generate(
    input_ids,
    max_new_tokens=100,   # Adjust this value to control the length of the output
    no_repeat_ngram_size=2,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    eos_token_id=tokenizer.eos_token_id
)

# Decode the generated tokens
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
print("Generated Story:\n")
print(generated_text)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
The new lm_head weights will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


OutOfMemoryError: CUDA out of memory. Tried to allocate 86.00 MiB. GPU 0 has a total capacity of 7.78 GiB of which 90.94 MiB is free. Including non-PyTorch memory, this process has 7.20 GiB memory in use. Of the allocated memory 7.01 GiB is allocated by PyTorch, and 92.85 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)